In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import os
import missingno as msno
import pickle
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import matplotlib
import random

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

from fbprophet import Prophet

import tensorflow as tf
from keras.layers import *
from keras import *
import keras

In [ ]:
orders = pd.read_csv('/content/drive/MyDrive/CJ_AI빅데이터_미래기술_챌린지/orders.csv')
deliveries = pd.read_csv('/content/drive/MyDrive/CJ_AI빅데이터_미래기술_챌린지/deliveries.csv')

def beg_end_month(x):
    if x<=10:
        return '월초'
    elif 10<x<=20:
        return '월중'
    elif 20<x<=31:
        return '월말'
orders.BKG_DATE = pd.to_datetime(orders.BKG_DATE, format='%Y-%m-%d')
orders.INS_DATE = pd.to_datetime(orders.INS_DATE, format='%Y-%m-%d')
orders["BKG_TIME"] = pd.to_datetime(orders["BKG_TIME"], format='%Y-%m-%d %H:%M:%S')
orders['BKG_WEEK'] = orders.BKG_DATE.dt.week
orders['BKG_MONTH2'] = orders.BKG_DATE.dt.day.map(beg_end_month)

In [ ]:
data = orders[orders.BKG_TYP==7][orders.CORP_ID=='KX007'].groupby(['BKG_DATE','BKG_HOUR'])['ITEM_QTY'].sum().reset_index()
data = data.append(pd.DataFrame(dict(zip(['BKG_DATE','BKG_HOUR','ITEM_QTY'],[(pd.to_datetime('2021-06-28'),pd.to_datetime('2021-06-28')), (4,5), (0,0)]))))
data = data.sort_values(['BKG_DATE','BKG_HOUR'])

comb_date = []
for date, hour, _ in data.values:
    comb_date.append(pd.to_datetime(f'{date.year}-{date.month}-{date.day} {hour}:00:00'))
data['DATE'] = comb_date
data = data.set_index('DATE').drop(columns=['BKG_DATE', 'BKG_HOUR']).rename(columns={'ITEM_QTY':'TARGET'})

In [ ]:
train_y = data.iloc[:-720].values
val_y = data.iloc[-720:-360].values
test_y = data.iloc[-360:].values

val_index = data.iloc[-720:-360].index
test_index = data.iloc[-360:].index

In [ ]:
real_ = data.iloc[-720:]

In [ ]:
res_p = pd.read_csv('restults_prophet_hol.csv')
res_s = pd.read_csv('results_sarima.csv')
res_d = pd.read_csv('results_dnn.csv')
res_m = pd.read_csv('results_ml.csv')

res_p['ds'] = pd.to_datetime(res_p['ds'])
res_s['DATE'] = pd.to_datetime(res_s['DATE'])
res_d['DATE'] = pd.to_datetime(res_d['DATE'])
res_m = res_m.set_index(res_p.ds).rename(columns={'0':'TARGET'})

In [ ]:
res_p.set_index('ds', inplace=True)
res_s.set_index('DATE', inplace=True)
res_d.set_index('DATE', inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
res_p.yhat.plot(ax=ax, color='blue', label='Prophet', alpha=.5)
res_s.preds.plot(ax=ax, color='yellow', label='SARIMA', alpha=.5)
res_d['ensemble'].plot(ax=ax, color='red', label='DNN', alpha=.5)
res_m['TARGET'].plot(ax=ax, color='purple', label='ML', alpha=.5)
real_.plot(ax=ax, color='k', label='real')
plt.legend(bbox_to_anchor=[1,1])
plt.grid()
plt.show()

In [ ]:
res_m.iloc[360:]['TARGET']

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,10))
res_p.iloc[:360].yhat.plot(ax=ax[0], color='blue', label='Prophet', alpha=.5)
res_s.iloc[:360].preds.plot(ax=ax[0], color='yellow', label='SARIMA', alpha=.5)
res_d.iloc[:360]['ensemble'].plot(ax=ax[0], color='red', label='DNN', alpha=.5)
res_m.iloc[:360]['TARGET'].plot(ax=ax[0], color='purple', label='ML', alpha=.5)
real_.iloc[:360].plot(ax=ax[0], color='k', label='real')
ax[0].set_title('Validation Set')
ax[0].legend()
ax[0].grid()
res_p.iloc[360:].yhat.plot(ax=ax[1], color='blue', label='Prophet', alpha=.5)
res_s.iloc[360:].preds.plot(ax=ax[1], color='yellow', label='SARIMA', alpha=.5)
res_d.iloc[360:]['ensemble'].plot(ax=ax[1], color='red', label='DNN', alpha=.5)
res_m.iloc[360:]['TARGET'].plot(ax=ax[1], color='purple', label='ML', alpha=.5)
real_.iloc[360:].plot(ax=ax[1], color='k', label='real')
ax[1].set_title('Test Set')
ax[1].legend()
ax[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
data.mean() + data.std()*3

In [ ]:
def ensemble(p, s, d):
    if str(type(p)).startswith('<class \'numpy'):
    p = pd.Series(p.flatten(), columns=['p'])
    if str(type(s)).startswith('<class \'numpy'):
    s = pd.Series(s.flatten(), columns=['s'])
    if str(type(d)).startswith('<class \'numpy'):
    d = pd.Series(d.flatten(), columns=['d'])
    x = pd.concat([p.reset_index(drop=True),s.reset_index(drop=True),d.reset_index(drop=True)], axis=1)
    x = x.rename(columns=dict(zip(x.columns, ['p','s','d'])))
    x2 = []
    for data in x.values:
    if data[2] >= 2500:
        x2.append(data[2])
    else:
        x2.append(data.mean())
    return x2

In [ ]:
res_e = pd.DataFrame(ensemble(res_p.yhat, res_s.preds, res_d.ensemble),columns=['preds']).set_index(real_.index)

fig, ax = plt.subplots(figsize=(20,5))
res_e['preds'].plot(ax=ax, color='red', label='DNN')
real_.plot(ax=ax, color='k', label='real')
plt.legend(bbox_to_anchor=[1,1])
plt.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,10))
res_e.iloc[:360]['preds'].plot(ax=ax[0], color='red', label='DNN')
real_.iloc[:360].plot(ax=ax[0], color='k', label='real')
ax[0].set_title('Validation Set')
ax[0].legend()
ax[0].grid()
res_e.iloc[360:]['preds'].plot(ax=ax[1], color='red', label='DNN')
real_.iloc[360:].plot(ax=ax[1], color='k', label='real')
ax[1].set_title('Validation Set')
ax[1].legend()
ax[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
res_s.iloc[360:]['preds'].plot(color='k', label='pred', figsize=(10,4))
real_.TARGET.iloc[360:].plot(color='pink', label='real')
plt.legend()
plt.show()

### Test Score

In [ ]:
mean_squared_error(res_p.iloc[360:], real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error(res_s.iloc[360:], real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error(res_d.ensemble.iloc[360:], real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_p.yhat.iloc[360:] + res_s.preds.iloc[360:])/2, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_p.yhat.iloc[360:] + res_d.ensemble.iloc[360:])/2, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_d.ensemble.iloc[360:] + res_s.preds.iloc[360:])/2, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_p.yhat.iloc[360:] + res_m.TARGET.iloc[360:])/2, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_s.preds.iloc[360:] + res_m.TARGET.iloc[360:])/2, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_d.ensemble.iloc[360:] + res_m.TARGET.iloc[360:])/2, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_p.yhat.iloc[360:] + res_s.preds.iloc[360:] + res_m.TARGET.iloc[360:])/3, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_p.yhat.iloc[360:] + res_d.ensemble.iloc[360:] + res_m.TARGET.iloc[360:])/3, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_d.ensemble.iloc[360:] + res_s.preds.iloc[360:] + res_m.TARGET.iloc[360:])/3, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error((res_d.ensemble.iloc[360:] + res_s.preds.iloc[360:] + res_p.yhat.iloc[360:] + res_p.yhat.iloc[360:])/4, real_.iloc[360:], squared=False)

In [ ]:
mean_squared_error(res_e['preds'].iloc[360:], real_.iloc[360:], squared=False)

## Results_Labor

In [ ]:
def cal_labor(x):
    a= x[:8]
    b = x[8:17]    
    c = x[17:25]
    labor_a = (a.sum())/800 + (a.sum())/800 + (a.sum())/1600
    labor_b = (b.sum())/800 + (b.sum())/800 + (b.sum())/1600
    labor_c = (c.sum())/800 + (c.sum())/800 + (c.sum())/1600
    return labor_a, labor_b, labor_c

In [ ]:
preds_x = []
for i in range(15):
    preds_x.append(res_d['ensemble'][-377:-17][24*i:24+24*i])

In [ ]:
labors = []
for labor in preds_x:
    labors.append(cal_labor(labor))

In [ ]:
res_labor.iloc[0].cumsum()

In [ ]:
res_labor = pd.DataFrame(labors, columns=['오전조','오후조','야간조'], index=[pd.to_datetime(f'2021-06-{i}') for i in range(15,30)]).round()
res_labor.plot.bar(figsize=(10,4) ,stacked=True)
for i,x in enumerate(res_labor.values):
    a,b,c = x.cumsum()
    plt.text(i, a, a, ha='center', va='top')
    plt.text(i, b, b-a, ha='center', va='top')
    plt.text(i, c, c-a, ha='center', va='top')
plt.show()